In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
df_train = pd.read_csv("/kaggle/input/titanic/train.csv")
df_test = pd.read_csv("/kaggle/input/titanic/test.csv")
df_submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
y = df_train["Survived"].values
X = df_train[["Pclass", "Sex", "SibSp", "Parch", "Embarked"]].copy()

print("y Shape:", y.shape)
print("X Shape:", X.shape)

y Shape: (891,)
X Shape: (891, 5)


In [5]:
embarked_mode = X["Embarked"].mode()[0]
X["Embarked"] = X["Embarked"].fillna(embarked_mode)

In [6]:
X = pd.get_dummies(X, columns=["Pclass", "Sex", "Embarked"])

In [7]:
X.head()

,SibSp,Parch,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,False,False,True,False,True,False,False,True
1,1,0,True,False,False,True,False,True,False,False
2,0,0,False,False,True,True,False,False,False,True
3,1,0,True,False,False,True,False,False,False,True
4,0,0,False,False,True,False,True,False,False,True


In [8]:
X = X.values

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
X_train.shape

(712, 10)

### RandomForest


In [11]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(
    n_estimators=2000, min_samples_leaf=10, oob_score=True, n_jobs=-1
)

In [12]:
clf_rf.fit(X_train, y_train)

RandomForestClassifier(min_samples_leaf=10, n_estimators=2000, n_jobs=-1,
                       oob_score=True)

In [13]:
pred = clf_rf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
report = classification_report(y_test, pred)
print("accuracy:", accuracy)
print(report)

accuracy: 0.7932960893854749
              precision    recall  f1-score   support

           0       0.78      0.90      0.84       105
           1       0.82      0.64      0.72        74

    accuracy                           0.79       179
   macro avg       0.80      0.77      0.78       179
weighted avg       0.80      0.79      0.79       179



### KNN


In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier

In [15]:
parameters = {
    "n_neighbors": range(1, 50),
    "weights": ["uniform", "distance"],
}
gridsearch = GridSearchCV(KNeighborsClassifier(), parameters)
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50),
                         'weights': ['uniform', 'distance']})

In [16]:
best_k = gridsearch.best_params_["n_neighbors"]
best_weights = gridsearch.best_params_["weights"]
bagged_knn = KNeighborsClassifier(n_neighbors=best_k, weights=best_weights)
bagging_model = BaggingClassifier(bagged_knn, n_estimators=500)
bagging_model.fit(X_train, y_train)

BaggingClassifier(estimator=KNeighborsClassifier(n_neighbors=12),
                  n_estimators=500)

In [17]:
pred = bagging_model.predict(X_test)
accuracy = accuracy_score(y_test, pred)
report = classification_report(y_test, pred)
print("accuracy:", accuracy)
print(report)

accuracy: 0.7877094972067039
              precision    recall  f1-score   support

           0       0.78      0.89      0.83       105
           1       0.80      0.65      0.72        74

    accuracy                           0.79       179
   macro avg       0.79      0.77      0.77       179
weighted avg       0.79      0.79      0.78       179



### XGBoost


In [18]:
import xgboost as xgb

In [19]:
import time

gbm_param_grid = {
    "objective": ["binary:logistic"],
    "learning_rate": [0.01, 0.1, 0.5, 0.9],
    "n_estimators": [200, 500],
    "subsample": [0.3, 0.5, 0.9],
    "colsample_bytree": [0.3, 0.7, 1],
    "max_depth": [2, 5, 10, 15],
}

gbm = xgb.XGBClassifier()

start = time.time()
grid_mse = GridSearchCV(
    estimator=gbm,
    cv=4,
    verbose=1,
    param_grid=gbm_param_grid,
    scoring="neg_mean_squared_error",
)
grid_mse.fit(X_train, y_train)
end = time.time()
print("training and finding best params took:", end - start, "s")

Fitting 4 folds for each of 288 candidates, totalling 1152 fits
training and finding best params took: 144.02591514587402 s


In [20]:
pred = grid_mse.predict(X_test)
accuracy = accuracy_score(y_test, pred)
report = classification_report(y_test, pred)
print("accuracy:", accuracy)
print(report)

accuracy: 0.7988826815642458
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       105
           1       0.83      0.65      0.73        74

    accuracy                           0.80       179
   macro avg       0.81      0.78      0.78       179
weighted avg       0.80      0.80      0.79       179



### AdaBoost


In [21]:
from sklearn.ensemble import AdaBoostClassifier

In [22]:
clf_ada = AdaBoostClassifier(n_estimators=1000, learning_rate=0.5)
clf_ada.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=0.5, n_estimators=1000)

In [23]:
pred = clf_ada.predict(X_test)
accuracy = accuracy_score(y_test, pred)
report = classification_report(y_test, pred)
print("accuracy:", accuracy)
print(report)

accuracy: 0.7821229050279329
              precision    recall  f1-score   support

           0       0.80      0.84      0.82       105
           1       0.75      0.70      0.73        74

    accuracy                           0.78       179
   macro avg       0.78      0.77      0.77       179
weighted avg       0.78      0.78      0.78       179

